<a href="https://colab.research.google.com/github/rabiyulfahimhasim786/google-keyword-suggestion-tool/blob/main/Google_Keyword_Research_and_development_using_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Keyword Research in Google Suggestion tool

The following tool is designed to help to optimise SEO by providing most of the potential keywords related to the given initial words. 

The tool works in various languages, at the end, the CSV file will be received.

# 1. Settings

#### The required packages and libraries.

In [ ]:
import pandas as pd
import requests
import json
import time
import string
import nltk
nltk.download('punkt')
!pip install stop_words
from stop_words import get_stop_words
from google.colab import files
%load_ext google.colab.data_table
from collections import Counter
from json import loads

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=c81cc899b66db1e4c8c4e60cd3b13ec1ea3b5c4bf98738d9ec801ff32a48ae0a
  Stored in directory: /root/.cache/pip/wheels/da/d8/66/395317506a23a9d1d7de433ad6a7d9e6e16aab48cf028a0f60
Successfully built stop_words


Fill in the field lang_code with your language code (e.g. en, fr, es, nl) and enter up to 5 seed keywords you want to use.



In [ ]:
#language code and keywords
lang_code="da"#@param {type:"string"}
keyword1="mobile app developement" #@param {type:"string"}
keyword2="ios app developemnt" #@param {type:"string"}
keyword3="android app development" #@param {type:"string"}
keyword4="" #@param {type:"string"}
keyword5="" #@param {type:"string"}

# 2. Code implementation

In [ ]:
#generate keyword list
keywords=[keyword1,keyword2,keyword3,keyword4,keyword5]
keywordlist = list(filter(None, keywords))
keywordlist

['mobile app developement', 'ios app developemnt', 'android app development']

In [ ]:
#Make a list of letters to use for Google Suggest
letterlist=[""]
letterlist=letterlist+list(string.ascii_lowercase)

In [ ]:
#Google Suggest for each combination of keyword and letter
keywordsuggestions=[]
for keyword in keywordlist: 
  for letter in letterlist :
    URL="http://suggestqueries.google.com/complete/search?client=firefox&hl="+str(lang_code)+"&q="+keyword+" "+letter
    headers = {'User-agent':'Mozilla/5.0'} 
    response = requests.get(URL, headers=headers) 
    result = json.loads(response.content.decode('utf-8'))
    keywordsuggest=[keyword,letter] 
    for word in result[1]:
      if(word!=keyword):
        keywordsuggest.append(word)
    time.sleep(1)
    keywordsuggestions.append(keywordsuggest)
#crearte a dataframe from this list
keywordsuggestions_df = pd.DataFrame(keywordsuggestions)

In [ ]:
#Rename columns of dataframe
columnnames=["Keyword","Letter"]
for i in range(1,len(keywordsuggestions_df.columns)-1):
  columnnames.append("Suggestion"+str(i))
keywordsuggestions_df.columns=columnnames

In [ ]:
#Make a list of all suggestions
allkeywords = keywordlist
for i in range(1,len(keywordsuggestions_df.columns)-1):
  suggestlist = keywordsuggestions_df["Suggestion"+str(i)].values.tolist()
  for suggestion in suggestlist:
    allkeywords.append(suggestion)

In [ ]:
#exclude stopwords and seed keywords from this list
stop_words=get_stop_words(lang_code)
wordlist=[]
seed_words=[]
for keyword in keywords:
   for seed_word in nltk.word_tokenize(str(keyword).lower()):
     if(len(seed_word)>0):
       seed_words.append(seed_word)
for keyword in allkeywords:
   words = nltk.word_tokenize(str(keyword).lower()) 
   #word tokenizer
   for word in words:
     if(word not in stop_words and word not in seed_words and len(word)>1):
      wordlist.append(word)

In [ ]:
#find the most common words in the suggestions
most_common_words= [word for word, word_count in Counter(wordlist).most_common(200)]

In [ ]:
#assign each suggestion to a common keyword
clusters=[]
for common_word in most_common_words:
    for keyword in allkeywords:
      if(common_word in str(keyword)):
         clusters.append([keyword,common_word])
clusterdf = pd.DataFrame(clusters,columns=['Keyword', 'Cluster'])

# 3. The Final Results 

The list of keywords will be generated that might consist of more than a thousand different keywords related to words and sentences, after implementing the tool, it is recommended to go to the google keywords planner so to sort the keywords. 

In [ ]:
#create dataframe wiht clusters en suggestions
clusterdf.to_csv("keywords_clustered.csv")
files.download("keywords_clustered.csv")
clusterdf

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Keyword,Cluster
0,mobile app development in python,in
1,mobile app development xamarin,in
2,ios app development getting started,in
3,ios app development using kotlin,in
4,ios app development on windows,in
...,...,...
3131,android app developer vacancy,dev
3132,android app development w3schools,dev
3133,android app development course with kotlin and...,a-z
3134,mobile app development kuwait,kuwait
